<a href="https://colab.research.google.com/github/leolaipelt/EarthEnginePythonBasico/blob/master/Visao_Geral_sobre_ImageCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visao Geral sobre ImageCollection

ImageCollection é o nome denominado para manipular uma série de imagens no EE. 

Podemos criar uma ImageCollection com o comando: *ee.ImageCollection()* ou a partir de imagens separadas com o comando *ee.ImageCollection.fromImages()*.

**Vamos conferir um exemplo indicado pelo site do EE:**

In [0]:
!pip install earthengine-api


In [0]:
import ee; ee.Authenticate()



In [0]:
ee.Initialize()

In [7]:
#Criando duas imagens arbitrárias e de valor constante

image1 = ee.Image(1)
image2 = ee.Image(2)

#Criando uma ImageCollection com as duas imagens anteriores

colecaodeimagens = ee.ImageCollection([image1, image2])
print(colecaodeimagens.getInfo())

#Usando fromImages().
colecaodeimagens_FromImages = ee.ImageCollection.fromImages([ee.Image(3), ee.Image(4)])

print(colecaodeimagens_FromImages.getInfo())

#E juntando duas colecoes de imagens
mergedCollection = colecaodeimagens.merge(colecaodeimagens_FromImages)

print(mergedCollection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], 'properties': {'system:index': '1'}}]}
{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 4, 'max': 4}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], '

 # Metadata e Informações da **ImageCollection** 

Ver as informações das imagens que fazem parte do grupo de ImageCollections é praticamente igual se estivessemos querendo saber uma informação de apenas uma imagem. Contudo, é necessário tomar cuidado com algumas limitações.

O EE limita o "print" das informações de uma ImageCollection se essa solicitação ultrapassar 5000 elementos! Ou seja, se a sua ImageCollection possui 5000 elementos, um erro de *"without memory"* ou similar irá aparecer!

Para evitar esse problema, é interessante a gente ser direto na solicitação de informação, além de utilizar as ferramentas que o EE disponibiliza para fazer isso.

**Vamos ver agora um exemplo de como utilizar o** *ee.Filter* na chamada das coleções de ImageCollection, para determinar, por exemplo, as órbitas-pontos dos satélites **LANDSAT**, ou a data específica que temos interesse!

In [14]:
#Carregando uma coleção de imagens do LANDSAT L8 e filtrando uma específica orbita-ponto

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
    .filter(ee.Filter.eq('WRS_PATH', 223))\
    .filter(ee.Filter.eq('WRS_ROW', 67))\
    .filterDate('2015-01-01', '2015-12-01');

#print('Collection: ', collection);

#Descobrindo Quantas imagens estão presentes nessa coleção
count = collection.size();
print(count.getInfo());

#O intervalo de datas das imagens!
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
#print('Date range: ', ee.Date(range.get('min')), ee.Date(range.get('max')))

#Obtendo alguma estatística como, por exemplo, A Elevação Solar na hora de passagem do satélite!
sunStats = collection.aggregate_stats('SUN_ELEVATION');
#print(sunStats.getInfo());

20


In [18]:
#Descobrir a imagem com menor cobertura de Nuvem dentro da Coleção
image = ee.Image(collection.sort('CLOUD_COVER').first()); #Sort orienta as imagens de menores valores a maiores (default) e em relação a propriedade escolhida
#Se usarmos .sort('CLOUD_COVER',False) estaremos ordenando na ordem decrescente
print('A porcentagem do dia com MENOR cobertura de núvem: ', image.get('CLOUD_COVER').getInfo());

image2 = ee.Image(collection.sort('CLOUD_COVER',False).first());
print('A porcentagem do dia com MAIS cobertura de núvem: ', image2.get('CLOUD_COVER').getInfo());


A porcentagem do dia com menos cobertura de núvem:  0.0
A porcentagem do dia com MAIS cobertura de núvem:  95.08


## O EE ainda possui outros tipos de filtros para localizarmos o que realmente queremos!

Um exemplo é o filtro para achar imagens dentro da coleção do LANDSAT que fazem parte da geometria que você escolheu - podendo ser um *Ponto, Retângulo, PolyLines....*


In [0]:
#Como entrada estou inserindo um ponto de interesse, e quero todas as imagens naquele período de tempo escolhido
collection_SR = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
  .filterDate('2018-01-01', '2018-12-31')
  .filterMetadata('CLOUD_COVER', 'less_than', 15)
  .filterBounds(ee.Geometry.Point(-51.0, -30.0)));  


#Aqui vamos montar uma image RGB  com a primeira imagem que está na lista da coleção

Mosaic=collection_SR.select('B4','B3','B2').first()

In [0]:
#Vamos visualizar esse produto com o mapa que já criamos!
import folium

def add_ee_layer(self,imagem,visParams, nome):
  mapID = ee.Image(imagem).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles ="https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Nosso Mapinha do Curso de Python",
    name = nome,
    overlay = True,
    control = True
  ).add_to(self)
  
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

In [121]:
# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(Mosaic, {'min': 0,'max': 2000,'gamma':'1.5'},'LANDSAT8')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

## Usando a Função *.map()*

Essa função é utilizada para realizar uma ação em todas as imagens que fazem parte da coleção. 

**Por exemplo**, se eu quiser o timestamp da imagem eu posso fazer o seguinte:

In [110]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
  .filter(ee.Filter.eq('WRS_PATH', 44))\
  .filter(ee.Filter.eq('WRS_ROW', 34));

def addtime(image):
  return image.addBands(image.metadata('system:time_start'));

print(collection.map(addtime));

ee.ImageCollection({
  "type": "Invocation",
  "arguments": {
    "collection": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "collection": {
              "type": "Invocation",
              "arguments": {
                "id": "LANDSAT/LC08/C01/T1_TOA"
              },
              "functionName": "ImageCollection.load"
            },
            "filter": {
              "type": "Invocation",
              "arguments": {
                "leftField": "WRS_PATH",
                "rightValue": 44
              },
              "functionName": "Filter.equals"
            }
          },
          "functionName": "Collection.filter"
        },
        "filter": {
          "type": "Invocation",
          "arguments": {
            "leftField": "WRS_ROW",
            "rightValue": 34
          },
          "functionName": "Filter.equals"
        }
      },
      "functionName": "Collection.fi

É utilizando o *.map()*, por exemplo, que podemos fazer uma filtragem das imagens **LANDSAT** em relação a presença de nuvens na cena!

Primeiro, precisamos criar uma função que utilize a banda *"pixel_qa"* que classifica os pixels na imagem para gente identificar os pixels referentes a nuvens (segundo o *pixel_qa*), e retirarmos de todas as bandas e de todas as imagens!

**Bora!**

In [0]:
#Exemplo de máscara para imagens LANDSAT8

def Cloud_MASKL8(image):
    
    quality = image.select('pixel_qa');
    c01 = quality.eq(322); #//Clear, low confidence cloud
    c02 = quality.eq(324); #//water, low confidence cloud
    c03 = quality.eq(1346);# //Clear terrain, terrain occluded
    mask = c01.Or(c02).Or(c03);
    return image.updateMask(mask);

In [126]:
#Vamos Aplicar naquela coleção que filtramos anteriormente!

Collection_cloudness=collection_SR.map(Cloud_MASKL8)

#NovaImagem
Mosaic_sem_nuvens=Collection_cloudness.select('B4','B3','B2').first()

meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(Mosaic_sem_nuvens, {'min': 0,'max': 2000,'gamma':'1.5'},'Sem Nuvens')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)